# OpenAI Assistant Agent
<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/agent/openai_assistant_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This shows you how to use our agent abstractions built on top of the [OpenAI Assistant API](https://platform.openai.com/docs/assistants/overview).


## Initial Setup 

Let's start by importing some simple building blocks.  

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


In [ ]:
!pip install llama-index

In [1]:
from llama_index.agent import OpenAIAssistantAgent

In [2]:
agent = OpenAIAssistantAgent.from_new(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    openai_tools=[{"type": "code_interpreter"}],
    instructions_prefix="Please address the user as Jane Doe. The user has a premium account."
)

In [4]:
agent._thread_id

'thread_kxFIsH2G10l4tG25HtpaewSk'

In [8]:
from openai import OpenAI

client = OpenAI()
messages = client.beta.threads.messages.list(
  thread_id="thread_kxFIsH2G10l4tG25HtpaewSk"
)
print(messages)

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_5nreRbfh68wEzNUgW4GSiXxP', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699315600, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_kxFIsH2G10l4tG25HtpaewSk')], object='list', first_id='msg_5nreRbfh68wEzNUgW4GSiXxP', last_id='msg_5nreRbfh68wEzNUgW4GSiXxP', has_more=False)


In [9]:
run = client.beta.threads.runs.create(
  thread_id=agent._thread_id,
  assistant_id=agent._assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

In [15]:
run = client.beta.threads.runs.retrieve(
  thread_id=agent._thread_id,
  run_id=run.id
)
print(run)

Run(id='run_YF4sQiF7RAd8ntrLZuHfK8qP', assistant_id='asst_1AzL2Slealt2GbtJKpDYapXt', cancelled_at=None, completed_at=None, created_at=1699315968, expires_at=None, failed_at=1699315980, file_ids=[], instructions='Please address the user as Jane Doe. The user has a premium account.', last_error=LastError(code='rate_limit_exceeded', message="We're currently processing too many requests - please try again later."), metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1699315968, status='failed', thread_id='thread_kxFIsH2G10l4tG25HtpaewSk', tools=[ToolAssistantToolsCode(type='code_interpreter')])


In [3]:
response = agent.chat("I need to solve the equation `3x + 11 = 14`. Can you help me?")

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_5nreRbfh68wEzNUgW4GSiXxP', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699315600, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_kxFIsH2G10l4tG25HtpaewSk')], object='list', first_id='msg_5nreRbfh68wEzNUgW4GSiXxP', last_id='msg_5nreRbfh68wEzNUgW4GSiXxP', has_more=False)
[ChatMessage(role=<MessageRole.USER: 'user'>, content='I need to solve the equation `3x + 11 = 14`. Can you help me?', additional_kwargs={'thread_message': ThreadMessage(id='msg_5nreRbfh68wEzNUgW4GSiXxP', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1699315600, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_kxFIsH2G10l4t

Exception: 

In [ ]:
print(str(response))